In [9]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
import datetime
import time

scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)
client = gspread.authorize(creds)
sheet = client.open("NU Confessions (Responses)").sheet1  # Open the spreadhseet
sheet2 = client.open("NU Confessions (Responses)").worksheet("Sheet2")

In [10]:
confNum = int(sheet2.acell("A1").value)
startPoint = int(sheet2.acell("B1").value)

print("Number of new confessions: "+str(len(sheet.get_all_values()) - startPoint+1))

Number of new confessions: 33


In [11]:
startPoint

5968

In [268]:
import gensim
import nltk
import numpy as np
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from pyarabic.araby import tokenize as arabic_tokenize

['School\\major',
 'Cs ',
 '',
 'Bmd',
 'Cs',
 'Bmd',
 'Cs',
 '',
 'كشري ابو طارق',
 'CS',
 'كمبيوتر انجينيرنج. و سي اس علي زبي',
 'Engineering',
 '',
 '',
 'Bdsm',
 'Computer science',
 'Engineering',
 '',
 '',
 '',
 'Business',
 'Engineering',
 'Computer Science',
 '',
 'Engineering',
 '',
 'Engineering',
 'Computer Science',
 'Computer Science',
 'Business',
 'Computer Science',
 'Computer Science',
 'Computer Science',
 '',
 '',
 '',
 'Engineering',
 'Engineering',
 'Computer Science',
 'Biotechnology',
 'Engineering',
 '',
 'Computer Science',
 'Computer Science',
 'Biotechnology',
 'Computer Science',
 'Computer Science',
 'Computer Science',
 'Biotechnology',
 '',
 'Computer Science',
 'Computer Science',
 'Biotechnology',
 'Biotechnology',
 '',
 '',
 'Engineering',
 'Business',
 '',
 'Engineering',
 'Business',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Business',
 'Biotechnology',
 'Engineering',
 'Engineering',
 'Engineering',


In [431]:
data = sheet.col_values(2)[1:]
Y = sheet.col_values(3)[1:]

def preprocess_text(text):
    # Tokenize English text
    tokens = word_tokenize(text, language='English')
    
    return tokens

# Tokenize and preprocess the mixed-language data
tokenized_data = [preprocess_text(text) for text in data]

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_data, vector_size=100, window=10, min_count=1, sg=1)

# Get embeddings for each text
embeddings = [np.mean([model.wv[word] for word in tokens if word in model.wv], axis=0) for tokens in tokenized_data]

embeddings = [embedding / np.linalg.norm(embedding) if np.linalg.norm(embedding) != 0 else embedding for embedding in embeddings]

c:\Users\eslam\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\eslam\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [328]:
# handle empty strings
for cnt, d, in enumerate(zip(embeddings,Y)):
    try:
        if d[0].shape[0] != 100:
            print(d[0].shape)

    except IndexError:
        embeddings.pop(cnt)
        Y.pop(cnt)
        print("did")

    if d[1] == '':
        embeddings.pop(cnt)
        Y.pop(cnt)
        print("done")

done
did
did
did
did
done
did


In [329]:
X= np.array(embeddings)

In [330]:
from sklearn.preprocessing import OneHotEncoder

# Create label encoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the text data
Y_encoded = encoder.fit_transform(np.array(Y).reshape(-1,1))

c:\Users\eslam\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [539]:
for cnt, i in enumerate(data):
    if i == ' ' or i == '':
        data.pop(cnt)
        Y.pop(cnt)
        print("off")


for cnt, i in enumerate(Y):
    if i == ' ' or i == '':
        data.pop(cnt)
        Y.pop(cnt)
        print("aff")

In [433]:
from sklearn.preprocessing import LabelEncoder

# Create label encoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the text data
Y_encoded = encoder.fit_transform(np.array(Y).reshape(-1,1))

c:\Users\eslam\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [444]:
Y[0] != ''

False

In [418]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense



# Tokenize the text
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(data)

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(data)

# Pad sequences to ensure consistent input shape
max_len = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

vocab_size = len(tokenizer.word_index) + 1

# Define the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=max_len),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(encoder.categories_[0]), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, 1026, 64)          2864192   
                                                                 
 flatten_18 (Flatten)        (None, 65664)             0         
                                                                 
 dense_75 (Dense)            (None, 32)                2101280   
                                                                 
 dense_76 (Dense)            (None, 64)                2112      
                                                                 
 dense_77 (Dense)            (None, 3)                 195       
                                                                 
Total params: 4967779 (18.95 MB)
Trainable params: 4967779 (18.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [419]:
# Train the model
model.fit(padded_sequences, Y_encoded, epochs=10, batch_size = 32, validation_split= 0.2)


Epoch 1/10
150/150 [==============================] - 11s 64ms/step - loss: 1.0214 - accuracy: 0.5278 - val_loss: 0.9516 - val_accuracy: 0.6072
Epoch 2/10
150/150 [==============================] - 8s 52ms/step - loss: 0.8869 - accuracy: 0.5902 - val_loss: 0.9124 - val_accuracy: 0.6172
Epoch 3/10
150/150 [==============================] - 8s 53ms/step - loss: 0.2089 - accuracy: 0.9434 - val_loss: 1.1327 - val_accuracy: 0.5154
Epoch 4/10
150/150 [==============================] - 8s 56ms/step - loss: 0.0356 - accuracy: 0.9914 - val_loss: 1.2324 - val_accuracy: 0.5254
Epoch 5/10
150/150 [==============================] - 9s 63ms/step - loss: 0.0181 - accuracy: 0.9958 - val_loss: 1.3285 - val_accuracy: 0.5838
Epoch 6/10
150/150 [==============================] - 9s 60ms/step - loss: 0.0134 - accuracy: 0.9960 - val_loss: 1.3886 - val_accuracy: 0.5396
Epoch 7/10
150/150 [==============================] - 10s 66ms/step - loss: 0.0118 - accuracy: 0.9965 - val_loss: 1.4437 - val_accuracy: 0.52